최종본: 실제 시연 때 사용한 코드.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from collections import Counter
#import serial
import time

predicted_numbers = []  # 예측된 숫자 문자열들을 저장할 리스트
prediction_count = 0  # 예측 횟수를 세는 카운터


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # 텐서를 펼칩니다
        x = self.fc(x)
        return F.log_softmax(x)

arduino = serial.Serial('COM3', 9600)  # Replace 'COM3' with your Arduino's port
time.sleep(2)  

# 모델 로드
model = Net()
PATH = 'C:\\gg\\'
model.load_state_dict(torch.load(PATH + 'model_for_RO_CNN.pt', map_location='cpu'))
model.eval()


def nothing(x):
    pass

cap = cv2.VideoCapture(1)

#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("Min Radius", "Trackbars", 16, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 20, 80, nothing)
cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, nothing)


if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            distance_threshold = cv2.getTrackbarPos("Distance Threshold", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            #number_counts = {}
            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if 19 < radius < 27:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
                        roi = roi.astype("float32") / 255.0
                        roi = np.expand_dims(roi, axis=0)
                        
                        roi_tensor = torch.from_numpy(roi).unsqueeze(0)
                        
                        output = model(roi_tensor)
                        _, predicted = torch.max(output, 1)
                        num = predicted.item()
                        digits.append((num, x))
                        
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
                digits.sort(key=lambda d: d[1])  # x 좌표에 따라 정렬
                grouped_digits = []
                group = [digits[0]]
                for i in range(1, len(digits)):
                    distance = digits[i][1] - digits[i-1][1]  
                    #print(f"Distance between digit {i-1} and {i}: {distance}")
                    if 30 < distance < 50:
                        group.append(digits[i])
                    else:
                        grouped_digits.append(group)
                        group = [digits[i]]
                grouped_digits.append(group)  # 마지막 그룹 추가
                
                # 세 자리 숫자 추출 및 출력
                for group in grouped_digits:                 
                    if len(group) == 3:  
                        three_digit_number = ''.join([str(d[0]) for d in group])
                        predicted_numbers.append(three_digit_number)
                        prediction_count += 1
                        
                        if prediction_count >= 120:  # 60번 예측이 완료되면
                            most_common_num = Counter(predicted_numbers).most_common(1)[0][0]
                            print("Most Frequent Result:", most_common_num)
                            predicted_numbers = []  
                            prediction_count = 0  
                            
                            nnum = int(most_common_num)
                            if 100 <= nnum <= 299:
                                print("1")
                                aaa = 1
                                arduino.write(str(aaa).encode())
                                time.sleep(19)
                            elif 300 <= nnum <= 499:
                                print("2")
                                bbb = 2
                                arduino.write(str(bbb).encode())
                                time.sleep(19)
                            elif 500 <= nnum <= 699:
                                print("3")
                                ccc = 3
                                arduino.write(str(ccc).encode())
                                time.sleep(19)
                            elif 700 <= nnum <= 899:
                                print("4")
                                ddd = 4
                                arduino.write(str(ddd).encode())
                                time.sleep(19)
                            #time.sleep(3)
                            
                        
                        
                        #time.sleep(5)
                        
                        
                        #arduino.write(three_digit_number.encode())
            
            except Exception as e:
                pass

            cv2.imshow("Video Capture", bin_img)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")
cap.release()
cv2.destroyAllWindows()    

최종본 코드 최적화 & 보고서 전용

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from collections import Counter
#import serial
import time


class DigitRecognitionSystem:
    def __init__(self, model_path, camera_index=0,): #arduino_port='COM3'):
        self.model = self.load_model(model_path)
        #self.arduino = serial.Serial(arduino_port, 9600)
        self.cap = cv2.VideoCapture(camera_index)
        self.predicted_numbers = []
        self.prediction_count = 0
        self.setup_trackbars()

        #time.sleep(2)  # Wait for Arduino initialization

    def load_model(self, model_path): 
        model = Net()
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        model.eval()
        return model

    def setup_trackbars(self):    # 트랙바 생성 >> 원하는 크기의 바운딩 박스의 최대 & 최소 찾기 위해서
        cv2.namedWindow("Trackbars")
        cv2.createTrackbar("Min Radius", "Trackbars", 16, 30, lambda x: None)
        cv2.createTrackbar("Max Radius", "Trackbars", 20, 80, lambda x: None)
        cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, lambda x: None)

    def process_frame(self):      # imshow로 전처리된 화면 띄우기
        ret, img = self.cap.read()
        if ret:
            processed_img = self.preprocess_image(img)
            self.detect_and_classify_digits(processed_img)
            cv2.imshow("Processed Image", processed_img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                return False  # 'q'가 눌리면 False 반환
            return True  # 그렇지 않으면 True 반환
        else:
            print("No Frame...")
            return False
 
    def preprocess_image(self, img): # 비디오의 프레임 전처리 ,threshold 프레임의 픽셀값 이진화(흑,백)
        
        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV) #110기준 더 어두우면 흰, 밝으면 검으로 이진화 
        
        return bin_img
        

    def detect_and_classify_digits(self, img): 
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            print("No contours detected")
        
        digits = []

        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            
            if min_radius < radius < max_radius:
                xs, xe = int(x-radius), int(x+radius)
                ys, ye = int(y-radius), int(y+radius)
                cv2.rectangle(img, (xs, ys), (xe, ye), (200, 0, 0), 1) # radius 조건에 맞는 애들만 바운딩박스 씌움
                
                if xs < 0 or ys < 0 or xe > img.shape[1] or ye > img.shape[0]: #무시
                    continue
                
                roi = img[ys:ye, xs:xe]
                
                if roi.size == 0: #무시
                    continue

                roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA) # mnist 모델에 맞게 리사이즈함
                roi = roi.astype("float32") / 255.0
                roi = np.expand_dims(roi, axis=0)

                roi_tensor = torch.from_numpy(roi).unsqueeze(0) #텐서로 변환함(?)

                output = self.model(roi_tensor) #모델 사용해서 예측함
                _, predicted = torch.max(output, 1)
                num = predicted.item()

                cv2.putText(img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0)) #바운딩 박스 위에 예측한 숫자 텍스트로 시각화
        digits.sort(key=lambda d: d[1])  
        grouped_digits = []
        group = [digits[0]] if digits else []

        for i in range(1, len(digits)):
            distance = digits[i][1] - digits[i-1][1]
            if 30 < distance < 50: #숫자의 중심간 거리에 맞는 것만 리스트에 담기
                group.append(digits[i])
            else:
                grouped_digits.append(group)
                group = [digits[i]]
        grouped_digits.append(group)  

        
        for group in grouped_digits:  # 리스트에 조건에 맞는 녀석들 3자리 모이면 아래 과정 실행
            if len(group) == 3:  
                three_digit_number = ''.join([str(d[0]) for d in group])
                self.predicted_numbers.append(three_digit_number)
                self.prediction_count += 1

                if self.prediction_count >= 120:   #120번 예측해서 그 중 1짱 찾기
                    most_common_num = Counter(self.predicted_numbers).most_common(1)[0][0]
                    self.handle_arduino_communication(int(most_common_num))
                    self.predicted_numbers = []  
                    self.prediction_count = 0      
            
                

    def run(self): # 얜 뭐야 ㅋ
        if self.cap.isOpened():
            while True:
                if not self.process_frame():
                    break
        else:
            print("Camera not opened...")

        self.cap.release()
        cv2.destroyAllWindows()

    def handle_arduino_communication(self, number): # 최종 출력 및 아두이노로 보내기 
        if 100 <= number <= 299:
            print("Sending command 1 to Arduino")
            #self.arduino.write(b'1')
            time.sleep(19)  # Wait for the Arduino to process the command
        elif 300 <= number <= 499:
            print("Sending command 2 to Arduino")
            #self.arduino.write(b'2')
            time.sleep(19)
        elif 500 <= number <= 699:
            print("Sending command 3 to Arduino")
            #self.arduino.write(b'3')
            time.sleep(19)
        elif 700 <= number <= 899:
            print("Sending command 4 to Arduino")
            #self.arduino.write(b'4')
            time.sleep(19)

class Net(nn.Module): # mnsit 모델 아무 쓰잘대기 없음 (내가 만든 모델 수 + 퍼온거 = 10개)
    def __init__(self): 
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # 텐서를 펼칩니다
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

# Usage example
model_path = 'C:\\gg\\model_for_RO_CNN.pt' 
digit_system = DigitRecognitionSystem(model_path)
digit_system.run()